<link rel="preconnect" href="https://fonts.gstatic.com">
<link href="https://fonts.googleapis.com/css2?family=Fira+Code&display=swap" rel="stylesheet">

### License

<p style="font-family: 'Fira Code', monospace; font-size: 1.2rem">
Copyright (C) 2022, Xilinx, Inc.<br>
Copyright (C) 2022-2024, Advanced Micro Devices, Inc.
<br><br>
Licensed under the Apache License, Version 2.0 (the "License");<br>
you may not use this file except in compliance with the License.<br><br>
You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0"?>http://www.apache.org/licenses/LICENSE-2.0</a><br><br>
Unless required by applicable law or agreed to in writing, software<br>
distributed under the License is distributed on an "AS IS" BASIS,<br>
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.<br>
See the License for the specific language governing permissions and<br>
limitations under the License.<br>
</p>


# ChipScoPy ILA and VIO Example


<img src="../img/api_overview.png" width="500" align="left">

## Description
This example demonstrates how to program and communicate with ILA (Integrated Logic Analyzer) and
VIO (Virtual IO) cores using the ChipScoPy Python API.


## Requirements
- Local or remote Xilinx Versal board, such as a VCK190
- Xilinx hw_server 2025.1 installed and running
- Xilinx cs_server 2025.1 installed and running
- Python 3.9 or greater installed
- ChipScoPy 2025.1 installed
- Jupyter notebook support installed - Please do so, using the command `pip install chipscopy[jupyter]`

## 1 - Initialization: Imports and File Paths

After this step,
- Required functions and classes are imported
- URL paths are set correctly
- File paths to example files are set correctly

In [1]:
import sys
import os
from chipscopy import get_design_files
from chipscopy import create_session, report_versions, delete_session

In [2]:
# Make sure to start the hw_server and cs_server prior to running.
# Specify locations of the running hw_server and cs_server below.
# The default is localhost - but can be other locations on the network.
CS_URL = os.getenv("CS_SERVER_URL", "TCP:localhost:3042")
HW_URL = os.getenv("HW_SERVER_URL", "TCP:localhost:3121")

# specify hw and if programming is desired
HW_PLATFORM = os.getenv("HW_PLATFORM", "vck190")
PROG_DEVICE = os.getenv("PROG_DEVICE", True)

# The get_design_files() function tries to find the PDI and LTX files. In non-standard
# configurations, you can put the path for PROGRAMMING_FILE and PROBES_FILE below.
design_files = get_design_files(f"{HW_PLATFORM}/production/chipscopy_ced")

PROGRAMMING_FILE = design_files.programming_file
PROBES_FILE = design_files.probes_file

print(f"HW_URL: {HW_URL}")
print(f"CS_URL: {CS_URL}")
print(f"PROGRAMMING_FILE: {PROGRAMMING_FILE}")
print(f"PROBES_FILE:{PROBES_FILE}")

HW_URL: TCP:localhost:3121
CS_URL: TCP:localhost:3042
PROGRAMMING_FILE: /scratch/wrk/chipscopy/chipscopy/examples/designs/vck190/production/chipscopy_ced/chipscopy_wrapper.pdi
PROBES_FILE:/scratch/wrk/chipscopy/chipscopy/examples/designs/vck190/production/chipscopy_ced/chipscopy_wrapper.ltx


## 2 - Create a session and connect to the hw_server and cs_server

The session is a container that keeps track of devices and debug cores.
After this step,
- Session is initialized and connected to server(s)
- Versions are detected and reported to stdout

In [3]:
session = create_session(cs_server_url=CS_URL, hw_server_url=HW_URL)
report_versions(session)

                      ChipScoPy Version Information                      
┌────────────────────────────┬──────────────────────────────────────────┐
│                     Entity │ Version                                  │
├────────────────────────────┼──────────────────────────────────────────┤
│                  ChipScoPy │ ┌───────────┬──────────────────────┐     │
│                            │ │ Attribute │ Value                │     │
│                            │ ├───────────┼──────────────────────┤     │
│                            │ │     Build │ 2025.1.1747162617    │     │
│                            │ │ Timestamp │ May 13 2025-11:56:57 │     │
│                            │ └───────────┴──────────────────────┘     │
│                            │                                          │
│ hw_server @ localhost:3121 │ ┌───────────┬───────────┐                │
│                            │ │ Attribute │ Value     │                │
│                            │ ├───────────┼───────────┤                │
│                            │ │    Status │ Connected │                │
│                            │ │     Build │ 0         │                │
│                            │ │   Version │ 2025.1    │                │
│                            │ │ Timestamp │           │                │
│                            │ └───────────┴───────────┘                │
│                            │                                          │
│ cs_server @ localhost:3042 │ ┌───────────────┬──────────────────────┐ │
│                            │ │     Attribute │ Value                │ │
│                            │ ├───────────────┼──────────────────────┤ │
│                            │ │        Status │ Connected            │ │
│                            │ │         Build │ 1747163727           │ │
│                            │ │       Version │ 2025.1.0             │ │
│                            │ │     Timestamp │ May 13 2025-12:15:27 │ │
│                            │ │       Package │ 2025.1.1747163727    │ │
│                            │ │ Artifact type │ pyinstaller          │ │
│                            │ └───────────────┴──────────────────────┘ │
└────────────────────────────┴──────────────────────────────────────────┘

## 3 - Program the device with the example design

After this step,
- Device is programmed with the example programming file

In [4]:
# Typical case - one device on the board - get it.
device = session.devices.filter_by(family="versal").get()
device.program(PROGRAMMING_FILE)

--> INFO: Programming device with: 
/scratch/wrk/chipscopy/chipscopy/examples/designs/vck190/production/chipscopy_ced/chipscopy_wrapper.pdi

Output()

## 4 - Discover Debug Cores

Debug core discovery initializes the chipscope server debug cores. This brings debug cores in the chipscope server online.

After this step,

- The cs_server is initialized and ready for use
- ILA and VIO core instances in the device are reported

In [5]:
device.discover_and_setup_cores(ltx_file=PROBES_FILE)
print(f"Debug cores setup and ready for use.")

Debug cores setup and ready for use.


In [6]:
# Print out the ILA core instance UUIDs and instance names
ila_cores = device.ila_cores
for index, ila_core in enumerate(ila_cores):
    print(f"{index} - {ila_core.core_info.uuid}   {ila_core.name}")

0 - C53C0D3572B655B6A8A13A86686D262D   chipscopy_i/noc_tg_bc/noc_bc_axis_ila_0
1 - D62FB110F1A25D439EB15D9149F450CE   chipscopy_i/counters/ila_fast_counter_0
2 - 9125D1429AD25223AECDD656C2856755   chipscopy_i/counters/ila_slow_counter_0


In [7]:
# Print out the VIO core instance UUIDs and instance names
vio_cores = device.vio_cores
for index, vio_core in enumerate(vio_cores):
    print(f"{index} - {vio_core.core_info.uuid}   {vio_core.name}")

0 - 021098F58FE659EFAD76657E635E53FE   chipscopy_i/noc_tg_bc/axis_vio_0
1 - 0E76DEFA789D564080CFA5332A88DB12   chipscopy_i/noc_tg_bc/noc_tg/inst/u_top_axi_mst/u_tg_top/gen_aximm_reg_space.u_reg_space/gen_vio_status_monitor.u_vio_status_monitor
2 - 718A03DBCD265D969EA5C15840AEFD17   chipscopy_i/counters/vio_fast_counter_0
3 - 22BC7ED6C4145AAB88C58EAEDFF2B04C   chipscopy_i/counters/vio_slow_counter_0


## 5 - VIO Control and ILA Capture

ILA and VIO are two important building blocks for debugging applications in hardware.
This example design design shows how to control IP using a VIO core and capture results with ILA.

In this Design,
- A VIO core controls the counter (reset, up/down, ce, load)
- An ILA core captures the counter values


<img src="img/capture_data.png" width="400" align="left">

In [8]:
# Grab the two cores we are interested in for the demonstration
# As shown above, a counter is connected to the ILA core.
# The VIO core controls the counter.

ila = device.ila_cores.get(name="chipscopy_i/counters/ila_slow_counter_0")
vio = device.vio_cores.get(name="chipscopy_i/counters/vio_slow_counter_0")

print(f"Using ILA: {ila.core_info.uuid}  {ila.name}")
print(f"Using VIO: {vio.core_info.uuid}  {vio.name}")

Using ILA: 9125D1429AD25223AECDD656C2856755  chipscopy_i/counters/ila_slow_counter_0
Using VIO: 22BC7ED6C4145AAB88C58EAEDFF2B04C  chipscopy_i/counters/vio_slow_counter_0


### 5a - Configure the counter using VIO output probes

<img src="img/vio_control_counter.png" width="300" align="left">

In [9]:
# Print all the VIO port and probe names. This is convenient to know which probes are connected to
# VIO ports. Also verifies probe names to pass to other functions.

print("VIO Port <---> Probe mapping:")
for probe in vio.probes:
    if probe.direction == "in":
        print(f"{probe.port_name} <-- {probe.probe_name}")
    else:
        print(f"{probe.port_name} --> {probe.probe_name}")

VIO Port <---> Probe mapping:
probe_in0 <-- chipscopy_i/counters/slow_counter_0_Q
probe_out0 --> chipscopy_i/counters/slow_counter_0_CE
probe_out1 --> chipscopy_i/counters/slow_counter_0_SCLR
probe_out2 --> chipscopy_i/counters/slow_counter_0_UP
probe_out3 --> chipscopy_i/counters/slow_counter_0_LOAD
probe_out4 --> chipscopy_i/counters/slow_counter_0_L


In [10]:
# Set up the VIO core to enable counting up from 0
#
vio.reset_vio()
vio.write_probes(
    {
        "chipscopy_i/counters/slow_counter_0_SCLR": 0,
        "chipscopy_i/counters/slow_counter_0_L": 0x00000000,
        "chipscopy_i/counters/slow_counter_0_LOAD": 0,
        "chipscopy_i/counters/slow_counter_0_UP": 1,
        "chipscopy_i/counters/slow_counter_0_CE": 1,
    }
)
print("Counter is now free-running and counting up")

Counter is now free-running and counting up


### 5b - Capture and display free-running counter using the ILA core

<img src="img/free_running_counter.png" width="350" align="left">

In [11]:
# Trigger ILA on the free running counter. Trigger set to the first time we see 0s in low 16-bits.
# This will show the counter is free running, and counting up

ila.reset_probes()
ila.set_probe_trigger_value("chipscopy_i/counters/slow_counter_0_Q_1", ["==", "0xXXXX_0000"])
ila.run_basic_trigger(window_count=1, window_size=32, trigger_position=16)
print("ILA is running - looking for trigger")

ILA is running - looking for trigger


In [12]:
# Wait for the ILA trigger with upload.
# Then print the captured ILA samples and mark the trigger position.

ila.wait_till_done(max_wait_minutes=0.1)
upload_successful = ila.upload()
if upload_successful:
    #
    # ila.waveform.get_data() returns data for probes. By default, all probes are included. But here we specify
    #     slow_counter_0_Q_1 so only the one probe data is returned.
    #
    samples = ila.waveform.get_data(
        ["chipscopy_i/counters/slow_counter_0_Q_1"],
        include_trigger=True,
        include_sample_info=True,
    )
    # Below is a convenient way to iterate over all probe values using a for loop.
    #
    # samples.values() is a list of lists including trigger, sample_index, window_index, window_sample_index,
    #     and any probes in samples from get_data() above.
    #
    # for trigger, sample_index, window_index, window_sample_index, probe0, probe1, ..., probeN in zip(*samples.values())
    #
    for trigger, sample_index, window_index, window_sample_index, slow_counter_0_Q_1 in zip(*samples.values()):
        trigger = "<-- Trigger" if trigger else ""
        print(
            f"Window:{window_index}  Window Sample:{window_sample_index}  {slow_counter_0_Q_1:10}  0x{slow_counter_0_Q_1:08X} {trigger}"
        )
else:
    print("Failed to upload ILA data from core")

Window:0  Window Sample:0   349831152  0x14D9FFF0 
Window:0  Window Sample:1   349831153  0x14D9FFF1 
Window:0  Window Sample:2   349831154  0x14D9FFF2 
Window:0  Window Sample:3   349831155  0x14D9FFF3 
Window:0  Window Sample:4   349831156  0x14D9FFF4 
Window:0  Window Sample:5   349831157  0x14D9FFF5 
Window:0  Window Sample:6   349831158  0x14D9FFF6 
Window:0  Window Sample:7   349831159  0x14D9FFF7 
Window:0  Window Sample:8   349831160  0x14D9FFF8 
Window:0  Window Sample:9   349831161  0x14D9FFF9 
Window:0  Window Sample:10   349831162  0x14D9FFFA 
Window:0  Window Sample:11   349831163  0x14D9FFFB 
Window:0  Window Sample:12   349831164  0x14D9FFFC 
Window:0  Window Sample:13   349831165  0x14D9FFFD 
Window:0  Window Sample:14   349831166  0x14D9FFFE 
Window:0  Window Sample:15   349831167  0x14D9FFFF 
Window:0  Window Sample:16   349831168  0x14DA0000 <-- Trigger
Window:0  Window Sample:17   349831169  0x14DA0001 
Window:0  Window Sample:18   349831170  0x14DA0002 
Window:0  W

### 5c - Trigger ILA using VIO Up/Down virtual switch

This step demonstrates how VIO and ILA can be combined to form powerful debug building blocks.

ILA is set to trigger when UP/DOWN counter signal edge rises or falls.
VIO drives the UP/DOWN counter control signal to 0 causing the counter to count down.
The signal transition causes ILA to trigger and capture data.

After this step,
- VIO drives counter to count from UP to DOWN
- ILA triggers on the UP to DOWN signal transition
- Waveform uploaded with the up/down trigger sample in the center of captured data


<img src="img/edge_trigger.png" width="550" align="left">

In [13]:
# Set ILA core to capture on a transition of the UP/DOWN toggle switch
# Once transition happens, trigger in the middle of the buffer.

ila.reset_probes()
ila.set_probe_trigger_value("chipscopy_i/counters/slow_counter_0_UP_1", ["==", "B"])
ila.run_basic_trigger(window_count=1, window_size=32, trigger_position=16)

print("ILA is running - looking for trigger")

ILA is running - looking for trigger


In [14]:
# VIO: Turn counter up/down switch to DOWN position.
# This will cause the running ILA to trigger on the transition edge from up to down.

vio.write_probes({"chipscopy_i/counters/slow_counter_0_UP": 0})

print("VIO changed up/down counter to count down")

VIO changed up/down counter to count down


In [15]:
# Print the captured ILA samples and mark the trigger position.
# Note that counter counts down after the trigger mark.

ila.wait_till_done(max_wait_minutes=0.1)
upload_successful = ila.upload()
if upload_successful:
    samples = ila.waveform.get_data(
        ["chipscopy_i/counters/slow_counter_0_Q_1"],
        include_trigger=True,
        include_sample_info=True,
    )
    for trigger, sample_index, window_index, window_sample_index, slow_counter_0_Q_1 in zip(*samples.values()):
        trigger = "<-- Trigger" if trigger else ""
        print(
            f"Window:{window_index}  Window Sample:{window_sample_index}  {slow_counter_0_Q_1:10}  0x{slow_counter_0_Q_1:08X} {trigger}"
        )
else:
    print("Failed to upload ILA data from core")

Window:0  Window Sample:0   390657743  0x1748F6CF 
Window:0  Window Sample:1   390657744  0x1748F6D0 
Window:0  Window Sample:2   390657745  0x1748F6D1 
Window:0  Window Sample:3   390657746  0x1748F6D2 
Window:0  Window Sample:4   390657747  0x1748F6D3 
Window:0  Window Sample:5   390657748  0x1748F6D4 
Window:0  Window Sample:6   390657749  0x1748F6D5 
Window:0  Window Sample:7   390657750  0x1748F6D6 
Window:0  Window Sample:8   390657751  0x1748F6D7 
Window:0  Window Sample:9   390657752  0x1748F6D8 
Window:0  Window Sample:10   390657753  0x1748F6D9 
Window:0  Window Sample:11   390657754  0x1748F6DA 
Window:0  Window Sample:12   390657755  0x1748F6DB 
Window:0  Window Sample:13   390657756  0x1748F6DC 
Window:0  Window Sample:14   390657757  0x1748F6DD 
Window:0  Window Sample:15   390657758  0x1748F6DE 
Window:0  Window Sample:16   390657759  0x1748F6DF <-- Trigger
Window:0  Window Sample:17   390657758  0x1748F6DE 
Window:0  Window Sample:18   390657757  0x1748F6DD 
Window:0  W

## 6 - Waveform Export - VCD (or CSV)

 Demonstrate how to export waveform data to a VCD file for visualizing waveform in other tools.

 Export includes complete waveform with probe, _TRIGGER, and _WINDOW.

 - For CSV export, substitute "CSV" for "VCD" argument.
 - To export to a file, substitute the filename for 'sys.stdout'

In [16]:
if upload_successful:
    ila.waveform.export_waveform("VCD", sys.stdout)

$date
        2025-05-29 14:38:20
$end
$version
        ChipScoPy Version 2025.1.1747162617
$end
$timescale
        1ps
$end
$scope module dut $end
$var reg 1 $ chipscopy_i/counters/slow_counter_0_CE_1  $end
$var reg 1 % chipscopy_i/counters/slow_counter_0_SCLR_1  $end
$var reg 1 & chipscopy_i/counters/slow_counter_0_UP_1  $end
$var reg 1 ' chipscopy_i/counters/slow_counter_0_LOAD_1  $end
$var reg 32 ( chipscopy_i/counters/slow_counter_0_L_1 [31:0] $end
$var reg 32 ) chipscopy_i/counters/slow_counter_0_Q_1 [31:0] $end
$var reg 1 * chipscopy_i/counters/ila_slow_counter_0_TRIG_OUT_trig  $end
$var reg 1 + chipscopy_i/counters/ila_fast_counter_0_TRIG_IN_ack  $end
$var reg 1 , chipscopy_i/counters/ila_fast_counter_0_TRIG_OUT_trig  $end
$var reg 1 - chipscopy_i/counters/ila_slow_counter_0_TRIG_IN_ack  $end
$var reg 16 . chipscopy_i/counters/slow_sine_Dout [15:0] $end
$var reg 16 / chipscopy_i/counters/slow_cosine_Dout [15:0] $end
$var reg 1 0 chipscopy_i/counters/locked  $end
$var reg 1 ! _T

In [17]:
## When done with testing, close the connection
delete_session(session)